In [56]:
import random
from bs4 import BeautifulSoup as bs
import time
import requests
import re
import numpy as np
import pandas as pd

In [57]:
url = requests.get('https://myanimelist.net/anime/season/archive')
soup = bs(url.text,"html5lib")


In [58]:
seasons = soup.find_all('td')

randindex = random.sample(range(0, 150), 96)
seasons_to_pick = [seasons[x] for x in randindex]
seasons_to_pick

[<td>\n                                              <a href="https://myanimelist.net/anime/season/2007/fall">\n                  Fall 2007\n                </a>\n                          </td>,
 <td>\n                                              <a href="https://myanimelist.net/anime/season/1998/fall">\n                  Fall 1998\n                </a>\n                          </td>,
 <td>\n                                              <a href="https://myanimelist.net/anime/season/2002/spring">\n                  Spring 2002\n                </a>\n                          </td>,
 <td>\n                                              <a href="https://myanimelist.net/anime/season/2012/spring">\n                  Spring 2012\n                </a>\n                          </td>,
 <td>\n                                              <a href="https://myanimelist.net/anime/season/2011/summer">\n                  Summer 2011\n                </a>\n                          </td>,
 <td>\n 

In [61]:
df = pd.DataFrame(columns=['Name','Studio','Episodes','Source','Type','Score','Year'])
"""fine tune the lookup list"""
def getText(soup):
    for s in soup.find_all('div',{'class':'seasonal-anime-list js-seasonal-anime-list js-seasonal-anime-list-key-1 clearfix'}):
        k = s.findAll('div',{'class':'seasonal-anime js-seasonal-anime'})
        for anime in k:
            print anime.find('a',{'class' : 'link-title'}).text.strip()
            print anime.find('span',{"class":'producer'}).text.strip()
            print anime.find('div',{'class':'eps'}).text.strip().split(' ')[0]
            print anime.find('span',{"class":'source'}).text.strip()
            print re.search("[a-zA-Z]+",str(anime.find('div',{'class':'info'}).text)).group(0)
            print anime.find('span',{'class':'score'}).text.strip()

In [13]:
time_avg = []
for index,s in enumerate(seasons_to_pick):
    t = time.time()
    try:
        url = requests.get(s.find('a').get('href'), timeout=.9,verify=True,stream=False)
        print '{} time to grab page is {}'.format(index,time.time() - t)
        time_avg.append(time.time() - t)
        soup = bs(url.text)
        getText(soup)
    except requests.exceptions.Timeout:
        print 'taking too long, skipping'
print np.average(time_avg)

0 time to grab page is 1.625
[<div class="seasonal-anime js-seasonal-anime" data-genre="4,8,22,23,36,37"><div>\n    <div class="title"><a class="icon-watch fl-r" href="https://myanimelist.net/anime/2167/Clannad/video" title="Watch Episode Video">Watch Video</a><p class="title-text">\n        <a class="link-title" href="https://myanimelist.net/anime/2167/Clannad">Clannad</a>\n      </p>\n    </div>\n\n    <div class="prodsrc">\n      <span class="producer"><a href="/anime/producer/2/Kyoto_Animation" title="Kyoto Animation">Kyoto Animation</a></span>\n      <div class="eps">\n                <a href="https://myanimelist.net/anime/2167/Clannad/episode"><span>23 eps</span>\n        </a>\n      </div>\n\n      <span class="source">Visual novel</span>\n\n      <a class="button_add btn-anime-watch-status js-anime-watch-status notinmylist" href="https://myanimelist.net/login.php?error=login_required&amp;from=%2Fanime%2Fseason%2F2007%2Ffall" title="Quick add anime to my list">add</a>\n    </div

info TV -
      
                  Apr 1, 2007
scormem 1,058
      
      
        7.09
title Taichi Senjimon
prodsrc Toei Animation
      
                39 eps
        
      

      -

      add
eps 39 eps
genres-inner js-genre-inner Action
      
        Adventure
      
        Fantasy
      
        Magic
info TV -
      
                  Apr 29, 2007
scormem 644
      
      
        6.56
title Ojarumaru
prodsrc Studio Gallop
      
                ? eps
        
      

      Original

      add
eps ? eps
genres-inner js-genre-inner Adventure
      
        Comedy
      
        Fantasy
info TV -
      
                  Oct 5, 1998, 18:10 (JST)
scormem 635
      
      
        6.42
title Revbahaf Wang-gug Jaegeon-soelgi
prodsrc -
      
                26 eps
        
      

      -

      add
eps 26 eps
genres-inner js-genre-inner Comedy
      
        Drama
      
        Shoujo
info TV -
      
                  May 21, 2007
scormem 274
      
      
        N/A
title L

2 time to grab page is 0.638000011444
[<div class="seasonal-anime js-seasonal-anime" data-genre="24,29,8,18"><div>\n    <div class="title"><p class="title-text">\n        <a class="link-title" href="https://myanimelist.net/anime/930/Densetsu_Kyojin_Ideon">Densetsu Kyojin Ideon</a>\n      </p>\n    </div>\n\n    <div class="prodsrc">\n      <span class="producer"><a href="/anime/producer/14/Sunrise" title="Sunrise">Sunrise</a></span>\n      <div class="eps">\n                <a href="https://myanimelist.net/anime/930/Densetsu_Kyojin_Ideon/episode"><span>39 eps</span>\n        </a>\n      </div>\n\n      <span class="source">Original</span>\n\n      <a class="button_add btn-anime-watch-status js-anime-watch-status notinmylist" href="https://myanimelist.net/login.php?error=login_required&amp;from=%2Fanime%2Fseason%2F1980%2Fspring" title="Quick add anime to my list">add</a>\n    </div>\n\n    <div class="genres js-genre" id="930">\n      <div class="genres-inner js-genre-inner"><span class

KeyboardInterrupt: 

In [22]:
lookup = ['title','prodsrc','eps','genres-inner js-genre-inner','info','scormem']
for s in soup.find_all('div',{'class':'seasonal-anime-list js-seasonal-anime-list js-seasonal-anime-list-key-1 clearfix'}):
    k = s.findAll('div',{'class':'seasonal-anime js-seasonal-anime'})
    for anime in k:
        for div in lookup:
            text = anime.find('div',{'class': div}).text
            print "\"{}\"".format(div)
            for t in text.split('\n'):
                print t.strip()

"title"
Densetsu Kyojin Ideon
"prodsrc"
Sunrise

39 eps



Original

add
"eps"
39 eps
"genres-inner js-genre-inner"
Sci-Fi

Space

Drama

Mecha
"info"
TV -

May 8, 1980
"scormem"
6,360


7.05
"title"
Tsurikichi Sanpei
"prodsrc"
Nippon Animation

109 eps



Manga

add
"eps"
109 eps
"genres-inner js-genre-inner"
Adventure

Sports
"info"
TV -

Apr 7, 1980, 19:00 (JST)
"scormem"
1,832


7.16
"title"
Uchuu Taitei God Sigma
"prodsrc"
-

50 eps



-

add
"eps"
50 eps
"genres-inner js-genre-inner"
Action

Space

Mecha
"info"
TV -

Mar 19, 1980
"scormem"
401


6.73
"title"
Watch Promotional Video
Muu no Hakugei
"prodsrc"
-

26 eps



Original

add
"eps"
26 eps
"genres-inner js-genre-inner"
Action

Fantasy

Sci-Fi
"info"
TV -

Apr 4, 1980, 18:00 (JST)
"scormem"
400


6.58
"title"
Moero Arthur: Hakuba no Ouji
"prodsrc"
Toei Animation

22 eps



-

add
"eps"
22 eps
"genres-inner js-genre-inner"
Adventure

Romance

Historical

Drama
"info"
TV -

Apr 6, 1980, 19:00 (JST)
"scormem"
313


6.33
"title"

In [72]:
t = '                  Oct 8, 1997              '
print t.strip()

Oct 8, 1997


In [59]:
for s in soup.find_all('div',{'class':'seasonal-anime-list js-seasonal-anime-list js-seasonal-anime-list-key-1 clearfix'}):
    k = s.findAll('div',{'class':'seasonal-anime js-seasonal-anime'})
    for anime in k:
        print anime.find('a',{'class' : 'link-title'}).text.strip()
        print anime.find('span',{"class":'producer'}).text.strip()
        print anime.find('div',{'class':'eps'}).text.strip().split(' ')[0]
        print anime.find('span',{"class":'source'}).text.strip()
        print re.search("[a-zA-Z]+",str(anime.find('div',{'class':'info'}).text)).group(0)
        print anime.find('span',{'class':'score'}).text.strip()

        

In [52]:
res = (re.search("[a-zA-Z]+",str(blah))).group(0)

In [53]:
res

'TV'